# BoolForge Tutorial #1: Working with Boolean functions

In this tutorial, we will explore the `BooleanFunction` class — the foundation of BoolForge.
Boolean functions form the regulatory rules in Boolean network models of gene regulation, so understanding their structure is essential before studying networks.

## What you will learn
*What you will learn:* In this tutorial you will:

- create Boolean functions from truth tables and from textual expressions,
- inspect core attributes such as degree, variable names, and stored properties,
- compute basic structural properties (essential variables, Hamming weight, bias),
- and convert Boolean functions into logical and polynomial representations and CANA objects.

---
## 0. Setup

In [ ]:
import boolforge
import numpy as np

---
## 1. Create a Boolean function
Boolean functions can be described in logical form, as polynomials, or as truth tables. BoolForge treats Boolean functions as binary vectors of length $2^n$, where n is the number of inputs. The vectors describe the right side of the truth table. The left side of the truth table is not stored because it is the same for any function with n inputs. For example, the function f(A,B) = A AND B is stored as [0,0,0,1], which is exactly the right side of the truth table
<div align="center">

| A | B | f(A,B) |
| :-: | :-: | :-: |
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

</div>

### 1.1 Create a Boolean function from truth table

An instance of `BooleanFunction` can be generated by specifying the right side of the truth table, i.e., by providing a binary vector of length $2^n$ for any $n\geq 0$. For example, to create the AND function above, we can write

In [ ]:
# Define a simple Boolean function f(A,B) = A AND B with optional name 'f_AND'
f = boolforge.BooleanFunction([0, 0, 0, 1], name="f_AND")
print('f',f)
print('truth table of f:\n',f.to_truth_table())

### 1.2 Create a Boolean function from text

Instances of `BooleanFunction` can also be created from text. For example, to define the same function as f, we can write

In [ ]:
f2 = boolforge.BooleanFunction('A and B')
print('f2',f2)

The text processor is fairly versatile. For example, we can define the same function as f also by writing

In [ ]:
f3 = boolforge.BooleanFunction('A + B > 1')
print('f3',f3)

Some examples of more complicated functions include

In [ ]:
# Define a 3-input function without any symmetries
g = boolforge.BooleanFunction('(A AND B) OR (NOT A AND C)')

# Define a 3-input linear / parity function 
h = boolforge.BooleanFunction('(x + y + z) % 2 == 0')

# Define a 3-input threshold function
k = boolforge.BooleanFunction('x + y - z > 0')

labels = ['g','h','k']
boolforge.display_truth_table(g,h,k,labels=labels)

### 1.3 Create Boolean functions by combining existing BooleanFunction objects

BoolForge also allows you to construct new Boolean functions by combining
existing ones using Boolean algebra operations.  
This is useful when building larger rules from simpler components.

Supported operations include:
- `+` for addition modulo 2
- `&` for logical AND  
- `|` for logical OR  
- `^` for XOR  
- `~` for NOT

Here are a few examples:

In [ ]:
a = boolforge.BooleanFunction("X + Y == 1")
b = boolforge.BooleanFunction("X OR Y")

# Negation: ¬A
not_a = ~a

# AND: A ∨ B
a_and_b = a & b

# AND: A ∧ B
a_or_b = a | b

# XOR: A ⊕ B
a_xor_b = a ^ b

labels = ['a','b','not a','a AND b','a OR b','a XOR b']
boolforge.display_truth_table(a,b,a_and_b,a_or_b,a_xor_b,not_a,labels=labels)

---
## 2. Attributes of BooleanFunction

Every instance of `BooleanFunction` has five attributes:
<div align="center">

| attribute | data type | description | 
| :-: | :-: | :- | 
| f | np.array(int) | stores the Boolean function (the right side of its truth table) |
| n | int | the degree, i.e., the number of variables |
| variables | np.array(str) | the name of the variables. By default, $x_0, \ldots, x_{n-1}$ | 
| name | str | optional, default '' |
| properties | dict | stores certain properties of the function as they are computed |

</div>

In [ ]:
print('f.f',f.f)
print('f.n',f.n)
print('f.variables',f.variables)
print('f.name',f.name)
print('f.properties',f.properties)

Since `f` was generated from truth table, its variables default to $x_0, x_1$. On the contrary, generating instances of `BooleanFunction` from text also provides the variable names. This becomes important when reading from text files entire Boolean networks, i.e., collections of Boolean functions. For example, for `f2`, `f3`, `g`, and `h`, we have:

In [ ]:
print('f2.variables',f2.variables)
print('f3.variables',f3.variables)
print('g.variables',g.variables)
print('h.variables',h.variables)

The variable order is determined by the first occurence of the variable in the generating text. See e.g.,

In [ ]:
print(boolforge.BooleanFunction('(x + y + z) % 2 == 0').variables)
print(boolforge.BooleanFunction('(y + z + x) % 2 == 0').variables)

The variable order determines how the truth table is indexed. For example, for variables [x,y,z], the entry in position i corresponds to the binary expansion of i over 
(x,y,z). Therefore, the same expression with a different variable order results in a different right-side truth table ordering. This becomes important when combining functions inside networks or importing networks from text files.

---
## 3. Basic properties of Boolean functions

We can inspect various properties of a Boolean function. The degree, i.e., the number of inputs, is readily available via 'f.n'. Other properties can be computed.

In [ ]:
print("Number of variables:", f.n)
print("Is constant?", f.is_constant())
print("Is degenerate?", f.is_degenerate())
print("Indices of essential variables:", f.get_essential_variables())
print("Type of inputs:", f.get_type_of_inputs())
print("Hamming weight:", f.get_hamming_weight())
print("Absolute bias:", f.get_absolute_bias())

Rerunning the above code for `g` helps understand the different properties. 
- 'g.is_constant()' checks if the function is constant, 
- 'g.is_degenerate()' checks if the function contains non-essential variables, 
- 'g.get_essential_variables()' provides the indices (Python: starting at 0!) of the essential variables, 
- 'g.get_type_of_inputs()' describes the type of each input ('increasing','decreasing','conditional', or 'non-essential').
- The Hamming weight is the number of 1s in the right side of the truth table.
- The absolute bias is $|\text{\#ones} - \text{\#zeros}| / 2^n$. It equals 1 for constant functions and 0 for unbiased functions.

In [ ]:
print("Number of variables:", g.n)
print("Is constant?", g.is_constant())
print("Is degenerate?", g.is_degenerate())
print("Indices of essential variables:", g.get_essential_variables())
print("Type of inputs:", g.get_type_of_inputs())
print("Hamming weight:", g.get_hamming_weight())
print("Absolute bias:", g.get_absolute_bias())

---
## 4. Convert to logical and polynomial expression

While Boolean functions are stored as truth tables, they can be expressed in logical and polynomial format.

In [ ]:
print(f"Logical form of {f.name}:", f.to_logical(AND=' ∧ ', OR=' ∨ ', NOT=' ¬'))
print(f"Polynomial form of {f.name}:", f.to_polynomial())

In addition, an instance of `BooleanFunction` can be turned into an instance of `BooleanNode` from the [CANA package](https:www.github.com). This requires the optional CANA package to be installed.

In [ ]:
cana_object = f.to_cana()
print(type(cana_object))

---
## 5. Summary of Key Concepts

Before moving on to more advanced topics, here is a short summary of the
fundamental ideas introduced in this tutorial:

### 5.1 Boolean functions
A Boolean function maps a set of binary inputs (0/1) to a single binary output.
BoolForge represents Boolean functions internally by their truth table, i.e.,
the list of outputs in lexicographic order of the input combinations.

### 5.2 Representations of Boolean functions
Boolean functions can be created from:
- a truth table (list of 0s and 1s),
- a logical expression written in Python syntax,
- algebraic combinations of existing BooleanFunction objects using operations such as  
  `|` (OR), `*` (AND), `^` (XOR), and other supported Boolean operations.

Each representation produces an equivalent internal truth-table-based object.

### 5.3 Variable names and ordering
BoolForge automatically infers variable names from the order of first appearance
in expressions.  
This order determines the indexing of the truth table and therefore affects how
the function interacts with larger Boolean networks.

### 5.4 Basic properties of Boolean functions
BoolForge can compute structural properties, including:
- the number of variables (`n`),
- the Hamming weight (number of 1s in the truth table),
- absolute bias (imbalance between 0s and 1s),
- essential and non-essential variables,
- positive/negative influence of each input.

These properties help characterize the function’s behavior and are used
throughout later tutorials.

### 5.5 Conversions and interoperability
BoolForge supports conversion between representations (e.g., truth table <-->
polynomial form) and is compatible with external packages such as **CANA** for
advanced analysis.  
This makes it easy to move between analytical frameworks and combine capabilities.

Together, these concepts provide the foundation for understanding canalization,
random Boolean function generation, and eventually the construction and analysis
of full Boolean networks.

---
## 6. Frequently Asked Questions (FAQ)

### 6.1 Why does the order of variables matter?
The order in which variables appear determines the ordering of the truth table.
For a function with variables `[A, B, C]`, the entry in position `i` corresponds
to the binary representation of `i` over `(A, B, C)`.  
If two equivalent expressions list variables in different orders, their truth
tables will be indexed differently.

To ensure reproducibility, always use consistent variable names and ordering.

### 6.2 How do I choose between defining a function via a truth table or via an expression?
Use a **truth table** if:
- you want full control over the outputs,
- you generated the table programmatically,
- or a biological dataset already provides the output values.

Use a **textual expression** if:
- the function has a natural logical description (e.g., `A and B`),
- you want readability,
- or you want BoolForge to automatically determine variables.

Both methods produce identical internal representations.

### 6.3 How do I control the variable names for a truth-table-defined function?
By default, BoolForge assigns names `x0, x1, ...`.

If you want custom variable names, specify them explicitly:

In [ ]:
boolforge.BooleanFunction([0,0,0,1], variables=["A", "B"])

### 6.4. Can I see all properties of a Boolean function at once?
Yes. The .summary() method prints all basic properties, while the properties dictionary lists precomputed and cached properties, such as various computation-intensive canalizing properties, discussed in detail in Tutorial #3:

In [ ]:
f = boolforge.BooleanFunction("(A and B) OR C")
#print(f"f.summary():\n {f.summary()}")

f.get_layer_structure()
print("f.properties:")
for key in f.properties:
    print(key,f.properties[key])

### 6.5 What is the difference between get_type_of_inputs() and monotonicity?
The method .get_type_of_inputs() identifies whether each input variable influences the
function positively, negatively, or not at all.
Monotonicity is a global property describing whether all inputs influence the
output in a consistent direction.
A function with at least one conditional input is thus non-monotone, even though some
 inputs may have clear influence types (either positive or negative).
 
### 6.6 Why do some functions have essential and non-essential variables?
A variable is *essential* if changing its value can change the output.
A variable is *non-essential* if the output never depends on it.
Example: `f(A,B) = A`. Here, B is non-essential and can be removed without changing the function.
When later creating Boolean networks, Boolean functions are simplified to remove any non-essential inputs.

### 6.7 What should I do if BoolForge raises an error when creating a function?
Most errors occur due to:
    
- incorrect truth-table length,
- non-binary entries,
- or syntax errors in expressions.

Check the “Common Errors” section above for guidance.
If the issue persists, verify that your Python expression is valid and that
variable names are consistent.

### 6.8 Why does BoolForge use Python boolean expressions instead of custom syntax?
This design choice:
    
- avoids learning a new grammar,
- leverages Python’s native evaluation,
- ensures compatibility with symbolic tools,
- and allows extremely flexible function definitions.

As long as the expression evaluates to 0/1 for all input combinations, it is valid.

### 6.9 Are Boolean functions in BoolForge intended only for biology?
No. Although BoolForge includes tools specifically inspired by gene regulatory
networks, Boolean functions appear across:
    
- biology & medicine (e.g., signaling pathways)
- engineering & computer science (e.g., digital logic design, circuit simplification, control theory),
- mathematics & theoretical modeling.

The tutorial keeps examples general so they remain broadly applicable.